In [2]:
##Copyright Thomas Paviot (tpaviot@gmail.com)
##and Andreas Plesch (@andreasplesch)
##
##This file is part of pythonOCC.
##
##pythonOCC is free software: you can redistribute it and/or modify
##it under the terms of the GNU Lesser General Public License as published by
##the Free Software Foundation, either version 3 of the License, or
##(at your option) any later version.
##
##pythonOCC is distributed in the hope that it will be useful,
##but WITHOUT ANY WARRANTY; without even the implied warranty of
##MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
##GNU Lesser General Public License for more details.
##
##You should have received a copy of the GNU Lesser General Public License
##along with pythonOCC.  If not, see <http://www.gnu.org/licenses/>.

import os
import re

from OCC.Core.TopoDS import TopoDS_Shell
from OCC.Core.BRep import BRep_Builder
from OCC.Core.XCAFDoc import (XCAFDoc_DocumentTool_ShapeTool,
                              XCAFDoc_DocumentTool_ColorTool)
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.XCAFDoc import (XCAFDoc_DocumentTool_ShapeTool,
                              XCAFDoc_DocumentTool_ColorTool,
                              XCAFDoc_DocumentTool_LayerTool,
                              XCAFDoc_DocumentTool_MaterialTool)

from OCC.Core.TDF import TDF_LabelSequence, TDF_Label
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Extend.TopologyUtils import TopologyExplorer, get_type_as_string
from OCC.Core.IFSelect import IFSelect_RetDone, IFSelect_ItemsByEntity

class DocFromSTEP:
    def __init__(self, stp_filename, doc_name=""):
        # create an handle to a document
        self._doc = TDocStd_Document(TCollection_ExtendedString(doc_name))

        step_reader = STEPCAFControl_Reader()
        step_reader.SetColorMode(True)
        step_reader.SetLayerMode(True)
        step_reader.SetNameMode(True)
        step_reader.SetMatMode(True)
        step_reader.SetGDTMode(True)
        print("read STP file", stp_filename, "...", end="")
        status = step_reader.ReadFile(stp_filename)
        if status == IFSelect_RetDone:
            step_reader.Transfer(self._doc)
            print("done.")
        else:
            raise IOError("failed")


    def get_doc(self):
        return self._doc


class SceneGrapheFromDoc:
    def __init__(self, doc, log=False):
        self._output_shapes = {}
        self._scene = []
        self._visited = {}
        self._uid_set = set()
        self._facesInSubshapes = set()

        self._shape_tool = XCAFDoc_DocumentTool_ShapeTool(doc.Main())
        self._color_tool = XCAFDoc_DocumentTool_ColorTool(doc.Main())

        self._locs = []
        self._log = log

        self._get_shapes()
    
    def get_scene(self):
        return self._scene
    
    def get_internalFaceEntries(self):
        return self._facesInSubshapes

    def _print_log(self, message):  # TODO: replace with the logging module functions
        if self._log:
            print(message)
        return


    def _get_shapes(self):
        labels = TDF_LabelSequence()
        self._shape_tool.GetFreeShapes(labels)

        self._print_log("Number of shapes at root : " + str(labels.Length()))

        for i in range(labels.Length()):
            root_item = labels.Value(i+1)
            self._get_sub_shapes(root_item, None, self._scene)


    def _get_sub_shapes(self, lab, loc, parent):

        labelString = lab.EntryDumpToString()
        if labelString in self._visited:
            return

        self._visited[labelString] = lab
        name = self._unescapeStep(lab.GetLabelName())

        if self._shape_tool.IsAssembly(lab):
            node = {'node' : 'Group',
                    'DEF' : labelString,
                    'name' : name,
                    'children' : []
                    }
            l_c = TDF_LabelSequence()
            self._shape_tool.GetComponents(lab, l_c)
            for i in range(l_c.Length()):
                label = l_c.Value(i + 1)
                #print("Group Name DEF :", name, labelString)    
                if self._shape_tool.IsReference(label):
                    self._print_log("########  component label :"+ self._unescapeStep(label.GetLabelName()) )
                    loc = self._shape_tool.GetLocation(label)
                    #print(" Transform  loc DEF          :", loc.HashCode(100))
                    label_reference = TDF_Label()
                    self._shape_tool.GetReferredShape(label, label_reference)
                    reference_name = self._unescapeStep(label_reference.GetLabelName())
                    self._print_log("########  Transform USE to DEF ==> referenced label : "+ reference_name )
                    trafo = {'node' : 'Transform',
                             'DEF' : label.EntryDumpToString(),
                             'name' : reference_name + '-trafo',
                             'transform' : loc,
                             'children': []
                            }
                    reference_entry = label_reference.EntryDumpToString()
                    if reference_entry in self._uid_set: #already defined, use USE
                        reference = {'node' : 'Transform',
                                     'USE' : reference_entry,
                                     'name' : reference_name + '-ref'
                                    }
                        trafo['children'].append(reference)
                    else:
                        self._uid_set.add(reference_entry)
                        self._get_sub_shapes(label_reference, loc, trafo['children'])
                    node['children'].append(trafo)

        elif self._shape_tool.IsSimpleShape(lab): # TODO recursive dive for subsubshapes
            #print("Transform DEF Shape Name :", name, labelString )
            shape = self._shape_tool.GetShape(lab)
            shape_type = get_type_as_string(shape)
            self._print_log(" #######  simpleshape of type " + shape_type + " for : " + name)

            c = self._set_color(lab, shape)
            clabel = self._color_tool.FindColor(c)
            clabelString = clabel.EntryDumpToString()

            #n = c.Name(c.Red(), c.Green(), c.Blue())
            #print('    instance color Name & RGB: ', n, c.Red(), c.Green(), c.Blue())

            labloc = self._shape_tool.GetLocation(lab)
            #print("    Shape Transform: ", labloc.HashCode(100))

            ##subshapes
            l_subss = TDF_LabelSequence()
            self._shape_tool.GetSubShapes(lab, l_subss)
            subcolorsUniform = True
            
            #always use Transform type for proper USE
            node = {'node': 'Transform',
                    'DEF': labelString,
                    'name': name,
                    'children': []
                    }
            
            shapenode = {'node' : 'Shape',
                         'label' : lab,
                         'shape' : shape,
                         'shapeType' : shape_type,
                         'name' : name + '-shape',
                         'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                         'color' : (c.Red(), c.Green(), c.Blue()),
                         'colorDEF' : clabelString
                        }
            
            #shapenode is attached below as needed

            if (l_subss.Length() == 0):
                
                node['name'] = node['name'] + '-wrapper'
                
                shapenode['name'] = name + '-singleshape'

            if (not labloc.IsIdentity()):
                
                node['transform'] = labloc
                
            for i in range(l_subss.Length()):

                lab_subs = l_subss.Value(i+1)
                shape_sub = self._shape_tool.GetShape(lab_subs)
                shape_type = get_type_as_string(shape_sub)
                #print("########  simpleshape subshape of type "+shape_type+" for :", name)
                #l_subsubss = TDF_LabelSequence()
                #self._shape_tool.GetSubShapes(lab_subs, l_subsubss)
                #print("########  subshape has subshapes: " + str(l_subsubss.Length()))
                #print("########  subshape has faces: ", len(solidfaces))
                #print("########  subshape has shells: ", expl.number_of_shells())

                c = self._set_color(lab_subs, shape_sub)
                clabel = self._color_tool.FindColor(c)
                clabelString = clabel.EntryDumpToString()
                n = c.Name(c.Red(), c.Green(), c.Blue())
                #print('    solidshape color RGB: ', c.Red(), c.Green(), c.Blue(), n)
                node_name = self._unescapeStep(lab_subs.GetLabelName())
                def_name = lab_subs.EntryDumpToString()
                subloc = self._shape_tool.GetLocation(lab_subs) # assume identity, otherwise we need another wrapper
                #print("    subshape Transform: ", subloc.HashCode(100))
                #default subshape
                subshapenode = {'node': 'SubShape',
                             'label': lab_subs,
                             'shape': shape_sub,
                             'shapeType': shape_type,
                             'DEF': def_name,
                             'name': node_name + '-subshape',
                             'colorString': f"{c.Red()} {c.Green()} {c.Blue()}",
                             'color': (c.Red(), c.Green(), c.Blue()),
                             'colorDEF': clabelString,
                             'trafo': subloc
                            }

                ### look for face colors
                expl = TopologyExplorer(shape_sub)
                solidfaces = list(expl.faces()) # works for all types
                #hasMultiColor = False
                if len(solidfaces) > 0:
                    colorFaceLists = {}
                    colorColors = {}
                    #print ("shapetype of solidface: ", get_type_as_string(solidfaces[0]))
                    facelabel = TDF_Label()
                    for i in range(0, len(solidfaces)):
                        found = self._shape_tool.FindSubShape(lab, solidfaces[i], facelabel)
                        if found:
                            self._facesInSubshapes.add(facelabel.EntryDumpToString())
                            c = self._set_color(facelabel, solidfaces[i])
                            clabel = self._color_tool.FindColor(c)
                            clabelString = clabel.EntryDumpToString()
                            if clabelString not in colorColors:
                                colorFaceLists[clabelString] = []
                            colorFaceLists[clabelString].append(solidfaces[i]) # collect face
                            colorColors[clabelString] = c # collect color
                    
                    # override default color, if only one color, is last color
                    clabel = self._color_tool.FindColor(c)
                    clabelString = clabel.EntryDumpToString()
                    subshapenode['colorString'] = f"{c.Red()} {c.Green()} {c.Blue()}"
                    subshapenode['color'] = (c.Red(), c.Green(), c.Blue())
                    subshapenode['colorDEF'] = clabelString

#                     for entry in iter(colorColors):
#                         c2 = colorColors[entry]
#                         print('    solidface color RGB: ', entry, c2.Red(), c2.Green(), c2.Blue())

                    # if more colors, make group with a shell per color (or compounds ?)
                    if len(list(colorFaceLists)) > 1:
                        
                        subshapenode = {'node' : 'Group',
                                     'label' : lab_subs,
                                     'shape' : shape_sub,
                                     'shapeType' : shape_type,
                                     'DEF' : def_name,
                                     'name' : node_name+'-colorshells',
                                     'children' : []
                                    }
                        f = 0
                        for entry in iter(colorFaceLists):
                            #make new shell with faces of single color
                            builder = BRep_Builder()
                            subshell = TopoDS_Shell() #use compound ?
                            builder.MakeShell(subshell)
                            faces = colorFaceLists[entry]
                            for i in range(0, len(faces)): # add faces
                                #print(entry,i,colorLists[entry][i])
                                builder.Add(subshell, faces[i])
                            #shape_type = get_type_as_string(subshell)
                            shape_type = "Shell"
                            c = colorColors[entry]
                            clabel = self._color_tool.FindColor(c)
                            clabelString = clabel.EntryDumpToString()
                            if (clabelString != shapenode['colorDEF']):
                                subcolorsUniform = False
                            shellnode = {'node' : 'SubShape',
                                         'label' : lab_subs,
                                         'shape' : subshell,
                                         'shapeType' : shape_type,
                                         'DEF' : f"{def_name}:{f}",
                                         'name' : node_name+'-colorshell',
                                         'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                                         'color' : (c.Red(), c.Green(), c.Blue()),
                                         'colorDEF' : clabelString
                                        }
                            subshapenode['children'].append(shellnode) #  add to group
                            f = f + 1
                        #hasMultiColor = True
                    #//end grouping into single color
                #//end face color check
                node['children'].append(subshapenode)
            #//end subshapes
            # only attach container shape if all face colors have the same color, required for buggy suspension
            # TODO: look for less heuristics
            if (subcolorsUniform):
                node['children'].append(shapenode)
        parent.append(node)

    def _set_color(self, lab, shape):
        #rint('is visible: ',color_tool.IsVisible(lab))
        c = Quantity_Color(0.5, 0.5, 0.5, Quantity_TOC_RGB)  # default color
        colorSet = False
        if (self._color_tool.GetInstanceColor(shape, 0, c) or
                self._color_tool.GetInstanceColor(shape, 1, c) or
                self._color_tool.GetInstanceColor(shape, 2, c)):

            colorSet = True

        if not colorSet:
            if (self._color_tool.GetColor(lab, 0, c) or
                    self._color_tool.GetColor(lab, 2, c) or
                    self._color_tool.GetColor(lab, 1, c)):

                colorSet = True

        if colorSet:
            self._color_tool.SetInstanceColor(shape, 0, c)
            self._color_tool.SetInstanceColor(shape, 1, c)
            self._color_tool.SetInstanceColor(shape, 2, c)

        return c

    def _unescapeStep(self, name):
        
        def _toUnicode(match):
            return chr(int(match.group(1), 16))

        reg1 = re.compile(r'\\X\\(..)')
        reg2 = re.compile(r'\\X2\\(....)\\X0\\')
        reg3 = re.compile(r'\\X4\\(........)\\X0\\')
        
        return reg3.sub(_toUnicode, reg2.sub(_toUnicode, reg1.sub(_toUnicode, name)))

if __name__ == "__main__":
    # test with the as1_pe.stp file
    # stp_filename = os.path.join('..', '..', '..', 'test', 'test_io', 'as1_pe_203.stp')
    stp_filename = os.path.join('assets','as1_pe_203.stp')
    doc_exp = DocFromSTEP(stp_filename)
    doc = doc_exp.get_doc()
    SceneGrapheFromDoc(doc, log=False)

read STP file assets/as1_pe_203.stp ...done.


In [3]:
import x3d.x3d as XX3D
from OCC.Core.gp import gp_XYZ, gp_Vec
from OCC.Core.Tesselator import ShapeTesselator
from OCC.Extend.TopologyUtils import is_edge, is_wire, discretize_edge, discretize_wire, get_type_as_string
import xml.etree.ElementTree as ET

def x3d_from_scenegraph(scene=[], facesInSolids=None, show_edges=True, edge_color=(0,0,0), log=False):#, doc=None):
    
    #shape_tool = XCAFDoc_DocumentTool_ShapeTool(doc.Main())
    
    appDEFset = set()
    if facesInSolids is None:
        facesInSolids = set()
        
    def print_log( message ):
        if (log):
            print (message)
        return
    
    def _x3dapplyLocation(x3dtransformnode, location):
            # get translation and rotation from location
            transformation = location.Transformation()
            rot_axis = gp_XYZ()
            #rot_angle = 0.0
            success, rot_angle = transformation.GetRotation(rot_axis)#.GetVectorAndAngle(rot_axis, rot_angle)
            translation = transformation.TranslationPart()
            scale_factor = transformation.ScaleFactor()
            x3dtransformnode.rotation = (rot_axis.X(), rot_axis.Y(), rot_axis.Z(), rot_angle)
            x3dtransformnode.translation = (translation.X(), translation.Y(), translation.Z())
            x3dtransformnode.scale = (scale_factor, scale_factor, scale_factor)
            return

    def _x3dgeofromTShape(shape):
        geo = XX3D.Box()
        #if label.IsNull():
        #    return geo
        #shape = shape_tool.GetShape(label)
        tesselator = ShapeTesselator(shape)
        tesselator.Compute(
            compute_edges=True,
            mesh_quality=1,
            parallel=True)
        nbr_edges = tesselator.ObjGetEdgeCount() # should be just one
        edge_point_set = []
        vertexCountList = []
        for i_edge in range(nbr_edges):
            nbr_vertices = tesselator.ObjEdgeGetVertexCount(i_edge)
            for i_vert in range(nbr_vertices):
                edge_point_set.append(tesselator.GetEdgeVertex(i_edge, i_vert))
            vertexCountList.append(nbr_vertices)
        mf_points = []
        for p in edge_point_set:
            mf_points.append( (p[0], p[1], p[2]) )
        coord = XX3D.Coordinate(point = mf_points)
        edges = XX3D.LineSet(coord = coord, vertexCount = vertexCountList)
        if ( is_edge(shape) or is_wire(shape) ):
            geo = edges
            edges = None
        else:
            x3dstring = tesselator.ExportShapeToX3DIndexedFaceSet()#x3dexp._triangle_sets[0] # there should be just one
            element = ET.XML(x3dstring)
            geo = XX3D.Box()
            if (element.tag == 'TriangleSet'):
                coordele = list(element.iter('Coordinate'))[0]
                mf_points = _MFVec3ffromString(coordele.attrib['point'])
                # can be empty
                if (len(mf_points) > 0):
                    coord = XX3D.Coordinate(point = mf_points)
                    normalele = list(element.iter('Normal'))[0]
                    mf_vectors = _MFVec3ffromString(normalele.attrib['vector'])
                    normal = XX3D.Normal(vector = mf_vectors)
                    geo = XX3D.TriangleSet(coord = coord, normal=normal, solid=False)
            # get tesselated triangleset or lineset
        return { 'x3dgeo': geo, 'x3dedges': edges }
    
    def _MFVec3ffromString(sepString):
        mflist = sepString.split()
        mf = []
        for i in range(len(mflist)):
            if (i % 3 == 2):
                mf.append((float(mflist[i-2]), float(mflist[i-1]), float(mflist[i])))
        return mf
    
    def _x3dappfromColor(c, DEFname, emissive):
        if (emissive):
            DEFname = DEFname + "-emissive"
        if DEFname in appDEFset:
            return XX3D.Appearance(USE = DEFname)
        else:
            appDEFset.add(DEFname)
            if (emissive):
                x3dmat = XX3D.Material(emissiveColor = c)
            else:
                x3dmat = XX3D.Material(diffuseColor = c, specularColor = (0.9, 0.9, 0.9), shininess = 1, ambientIntensity = 0.1)
            return XX3D.Appearance(DEF = DEFname, material = x3dmat)
    
    def _getx3dnode(node):
        
        def _sanitizeDEF(name):
# IdFirstChar ::=
# Any ISO-10646 character encoded using UTF-8 except: 0x30-0x3a, 0x0-0x20, 0x22, 0x23, 0x27, 0x2b, 0x2c, 0x2d, 0x2e, 0x5b, 0x5c, 0x5d, 0x7b, 0x7d, 0x7f ;
# first no [0-9],space,",#,',+,comma,-,.,[,\,],{,}
# IdRestChars ::=
# Any number of ISO-10646 characters except: 0x0-0x20, 0x22, 0x23, 0x27, 0x2c, 0x2e, 0x3a, 0x5b, 0x5c, 0x5d, 0x7b, 0x7d, 0x7f ;
# rest no space,",#,',comma,.,:,[,\,],{,}
            return 'L-' + ( name
                          .replace(" ","_")
                          .replace('"','^')
                          .replace('#','N')
                          .replace("'","^")
                          .replace(",",";")
                          .replace(".",";")
                          .replace(":","-")
                          .replace("[","(")
                          .replace("]",")")
                          .replace("{","(")
                          .replace("}",")")
                          .replace("\\","/") )

        def _applyDEFUSE(x3dnode):
            if 'USE' in node:
                x3dnode.USE = _sanitizeDEF(node['USE'])
            if 'DEF' in node:
                x3dnode.DEF = _sanitizeDEF(node['DEF'])
            return
        
        def _applychildren(x3dnode):
            if 'children' in node:
                for n in node['children']:
                    success, childlist = _getx3dnode(n)
                    if success:
                        x3dnode.children.extend(childlist)
            return
        
        if not 'node' in node:
            print_log('no node type, skipping')
            return False, None
        
        if 'DEF' in node:
            if node['DEF'] in facesInSolids:
                #print('in skipped list')
                return False, None
        
        ntype = node['node']
        #x3dnodelist = []
        edgeNode = None
        
        if (ntype == 'Group'):
            x3dnode = XX3D.Group(class_ = node['name'], children = [])
            _applyDEFUSE(x3dnode)
            _applychildren(x3dnode)
        
        elif (ntype == 'Transform'):
            x3dnode = XX3D.Transform(class_ = node['name'], children = [])
            if 'transform' in node:
                _x3dapplyLocation(x3dnode, node['transform'])
            _applyDEFUSE(x3dnode)
            _applychildren(x3dnode)
        
        elif (ntype == 'Shape' or ntype == 'SubShape'):
            shape_type = node['shapeType']
#             if (shape_type != "Solid" and shape_type != "Shell"):
#                 return False, None
            x3dnode = XX3D.Shape (class_ = node['name'])
            _applyDEFUSE(x3dnode)
            if 'shape' in node:
                shape = node['shape']
                geometryDict = _x3dgeofromTShape(shape)
                x3dnode.geometry = geometryDict['x3dgeo']
                isEdgeOrWire = is_edge(shape) or is_wire(shape)
                if 'color' in node:
                    colorDEF = _sanitizeDEF(node['colorDEF'])
                    x3dnode.appearance = _x3dappfromColor(node['color'], colorDEF, isEdgeOrWire)
                if (not isEdgeOrWire):
                    edgeNode = XX3D.Shape (class_ = node['name']+'-edge', visible = show_edges)
                    edgeNode.geometry = geometryDict['x3dedges']
                    edgeNode.appearance = _x3dappfromColor(edge_color, 'app-faceedge', True)
        else:
            print_log ('unknown node: --' + ntype + "--")
        
        x3dnodelist = [x3dnode] 
        if edgeNode is not None:
            x3dnodelist.append(edgeNode)
        return True, x3dnodelist

    x3dscene = XX3D.Scene(children=[])
    x3ddoc = XX3D.X3D(Scene = x3dscene)
    
    for node in scene:
        success, x3dnodelist = _getx3dnode(node)
        if success:
            x3dscene.children.extend(x3dnodelist)
        else:
            print_log ('no x3d for root node, skipped')
    
    return x3ddoc.XML()


x3d.py package loaded, have fun with X3D Graphics!


In [4]:
#print(x3dXML)

In [5]:
def x3dXML_to_x3domHTML(x3dXML):
    x3domHEAD = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''
    x3dele = list(ET.XML(x3dXML).iter('X3D'))[0]
    next(x3dele.iter('Scene')).append(ET.XML('<Environment gammaCorrectionDefault="none"/>'))
    x3dHTML = ET.tostring(x3dele, encoding="unicode", short_empty_elements=False)
    x3dHTML = x3dHTML.replace("visible=",'render=')
    return x3domHEAD + x3dHTML                             

In [6]:
stp_filename = os.path.join('assets','as1_pe_203.stp')
#stp_filename = os.path.join('assets','as1-oc-214.stp')
#stp_filename = os.path.join('assets','KR600_R2830-4.stp')
#stp_filename = os.path.join('assets','RC_Buggy_2_front_suspension.stp')
doc_exp = DocFromSTEP(stp_filename)
doc = doc_exp.get_doc()
scenegraph = SceneGrapheFromDoc(doc, log=True)
x3dXML = x3d_from_scenegraph(scenegraph.get_scene(), scenegraph.get_internalFaceEntries())


read STP file assets/as1_pe_203.stp ...done.
Number of shapes at root : 1
########  component label :PLATE
########  Transform USE to DEF ==> referenced label : PLATE
########  component label :=>[0:1:1:3]
########  Transform USE to DEF ==> referenced label : SOLID
 #######  simpleshape of type Solid for : SOLID
########  component label :=>[0:1:1:4]
########  Transform USE to DEF ==> referenced label : COMPOUND
 #######  simpleshape of type Compound for : COMPOUND
########  component label :L_BRACKET_ASSEMBLY
########  Transform USE to DEF ==> referenced label : L_BRACKET_ASSEMBLY_ASM
########  component label :L-BRACKET
########  Transform USE to DEF ==> referenced label : L-BRACKET
########  component label :=>[0:1:1:7]
########  Transform USE to DEF ==> referenced label : SOLID
 #######  simpleshape of type Solid for : SOLID
########  component label :=>[0:1:1:8]
########  Transform USE to DEF ==> referenced label : COMPOUND
 #######  simpleshape of type Compound for : COMPOUND
###

In [7]:
from IPython.display import HTML
HTML(x3dXML_to_x3domHTML(x3dXML))